## SageMaker SDK のバージョンを最新にしておく

In [ ]:
pip install sagemaker -U

In [ ]:
import sagemaker
print(sagemaker.__version__)

## 1-1. Hello SageMaker Training!
AWS のマネジメントコンテナ(AWS Deep Learning Container / SageMaker Scikit-learn Container) で Training Job を実行する
各マネジメントコンテナ用のクラスが SageMaker SDK に準備されていて、その class をインスタンス化( `estimator` 変数でインスタンス化する)する際に引数でバージョンを指定することで利用するコンテナが確定する

|  class  |  コンテナ  | コンテナ詳細 | 
| ---- | ---- | ---- | 
|  [`sagemaker.tensorflow.TensorFlow`](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#sagemaker.tensorflow.estimator.TensorFlow)  |  TensorFlow | [利用可能なバージョン一覧及びソース](https://github.com/aws/deep-learning-containers/blob/master/available_images.md) | 
|  [`sagemaker.pytorch.PyTprch`](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/sagemaker.pytorch.html#sagemaker.pytorch.estimator.PyTorch)  |  PyTorch  |^|
|  [`sagemaker.pytorch.MXNet`](https://sagemaker.readthedocs.io/en/stable/frameworks/mxnet/sagemaker.mxnet.html#sagemaker.mxnet.estimator.MXNet)  |  MXNet  |^|
|  [`sagemaker.pytorch.HuggingFace`](https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html#sagemaker.huggingface.HuggingFace)  |  HuggingFace  |^|
|  [`sagemaker.sklearn.SKLearn`](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html)  |  scikit-learn  |[コンテナソース](https://github.com/aws/sagemaker-scikit-learn-container)|


### 用意したコードの確認
`Hello SageMaker Training` という文字列を出力して終えるだけの簡単なスクリプトを予め準備

In [ ]:
!pygmentize ./src/1-1/hello_sagemaker_training.py

#### 書き換えたい人向け

In [ ]:
# %%writefile ./src/1-1/hello_sagemaker_training.py
# print('Hello SageMaker Training')
# exit()

### TensorFlow コンテナで実行
各フレームワーク(TensorFlow, PyTorch, MXNet, HuggingFace, scikit-learn) 毎に用意された estimator class で ジョブを定義し、estimator インスタンスを生成する。
* `entry_point` 引数に用意したコードを指定することで使える
* フレームワーク毎のクラス + py_version + framework_version + instance_type で使用するコンテナイメージが確定する。
  * 以下の場合は python3.8 が入った TensorFlow が 2.7.1 の CPU に最適化されたコンテナイメージ  
    763104351884.dkr.ecr.{REGION}.amazonaws.com/tensorflow-training:2.7.1-cpu-py38-ubuntu20.04-sagemaker
  * estimator インスタンスの `training_image_uri` メソッドでコンテナイメージの URI を確認できる
* estimator 生成時に `image_uri` 引数を指定することで直接コンテナイメージを指定することもできる

In [ ]:
# TensorFlow コンテナで Training Job
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
    entry_point='./src/1-1/hello_sagemaker_training.py',
    py_version='py38', 
    framework_version='2.7.1',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
print(f'トレーニングに使用するコンテナイメージは {estimator.training_image_uri()} です')
estimator.fit()

### PyTorch コンテナで実行
他のフレームワークでも使い方は全く一緒

In [ ]:
# PyTorch コンテナで Training Job
from sagemaker.pytorch import PyTorch
estimator = PyTorch(
    entry_point='./src/1-1/hello_sagemaker_training.py',
    py_version='py38', 
    framework_version='1.9.1',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit()

### MXNet コンテナで実行

In [ ]:
# MXNet コンテナで Training Job
from sagemaker.mxnet import MXNet
estimator = MXNet(
    entry_point='./src/1-1/hello_sagemaker_training.py',
    py_version='py37', 
    framework_version='1.8.0',
    instance_count=1,
    instance_type='ml.m4.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit()

### HuggingFace コンテナで実行

In [ ]:
# HuggingFace コンテナで　Training Job
# 注意 : GPU のクオータ緩和をしないと使えません
# from sagemaker.huggingface import HuggingFace
# estimator = HuggingFace(
#     entry_point='./src/1-1/hello_sagemaker_training.py',
#     py_version='py37', 
#     transformers_version='4.6.1',
#     tensorflow_version='2.4.1',
#     instance_count=1,
#     instance_type='ml.g4dn.xlarge',
#     role=sagemaker.get_execution_role()
# )
# estimator.fit()

### scikit-learn コンテナで実行

In [ ]:
# scikit-learn コンテナで Training Job
from sagemaker.sklearn import SKLearn
estimator = SKLearn(
    entry_point='./src/1-1/hello_sagemaker_training.py',
    py_version='py3', 
    framework_version='0.23-1',
    instance_count=1,
    instance_type='ml.c5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit()

### ジョブの実行結果確認
使用したコンテナイメージ URI や実行時間、使ったコードのありかなどが記録される

In [ ]:
# ジョブ定義や実行結果を確認
estimator.latest_training_job.describe()

### イメージ URI を指定してトレーニングを実行する場合

In [ ]:
from sagemaker.sklearn import SKLearn
estimator = SKLearn(
    entry_point='./src/1-1/hello_sagemaker_training.py',
    image_uri = estimator.latest_training_job.describe()['AlgorithmSpecification']['TrainingImage'],
    instance_count=1,
    instance_type='ml.c5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit()

## 1-2. データをトレーニングインスタンスに持ち込む
### 1-2-1. 単一のファイルを持ち込む
#### 持ち込むデータの確認
算数の計算問題を記載したデータを用意

In [ ]:
with open('./data/1-2-1/calc.txt','rt') as f:
    input_text_lines = f.read()
print('---データの確認---')
print(input_text_lines)
print('---計算結果---')
for input_text in input_text_lines.split('\n'):
    print(eval(input_text))

##### 書き換えたい方向け

In [ ]:
# text = """3+4
# 4-2
# 5*1
# 6/2"""
# with open('./data/1-2-1/calc.txt','wt') as f:
#     f.write(text)

#### 持ち込むデータを S3 にアップロード
`upload_data` メソッドを使うと S3 にデータをアップロードできる。返り値は S3 の URI

In [ ]:
# S3 にデータをアップロード
import sagemaker
input_s3_uri = sagemaker.session.Session().upload_data(path='./data/1-2-1/calc.txt', bucket=sagemaker.session.Session().default_bucket(), key_prefix='training/1-2-1')
print(input_s3_uri)

#### 用意したコードを確認
* トレーニングインスタンスに連携するデータはデフォルトだと環境変数 `SM_CHANNEL_TRAINING`(=`/opt/ml/input/training`) の値が示すディレクトリに格納される
* SM_CHANNEL_TRAINING にあるファイルを読み込み、1 行ずつデータを読み込んで文字列を数式として解釈して演算するコードを準備

In [ ]:
!pygmentize ./src/1-2-1/calc.py

#### トレーニングジョブの実行
データをトレーニングインスタンスに持ち込む場合は fit メソッドの引数に持ち込みたい S3 URI を指定指定

In [ ]:
# トレーニングジョブの実行
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
    entry_point='./src/1-2-1/calc.py',
    py_version='py38', 
    framework_version='2.7.1',
    instance_count=1,
    instance_type='local',
    role=sagemaker.get_execution_role()
)
estimator.fit(input_s3_uri)

### 1-2-2. ディレクトリごとトレーニングインスタンスに持ち込む
#### 持ち込むデータの確認
算数の計算問題が書かれたデータを同じディレクトリに 2 つファイルを準備

In [ ]:
FILE_NAME = './data/1-2-2/calc1.txt'
print(f'check {FILE_NAME}')
with open(FILE_NAME) as f:
    print(f.read())

FILE_NAME = './data/1-2-2/calc2.txt'
print(f'check {FILE_NAME}')
with open(FILE_NAME) as f:
    print(f.read())


##### 書き換えたい方向け

In [ ]:
# text1 = """2+2
# 3-1"""
# text2 = """5*1
# 16/2"""
# with open('./data/1-2-2/calc1.txt','wt') as f:
#     f.write(text1)
# with open('./data/1-2-2/calc2.txt','wt') as f:
#     f.write(text2)

#### 実行コードの確認
複数ファイルに対応できるよう for 文で SM_CHANNEL_TRAINING が示すディレクトリを順繰りに処理する

In [ ]:
!pygmentize ./src/1-2-2/calc.py

### トレーニングジョブの実行
* upload_data でディレクトリを指定するとそのディレクトリの全ファイルを S3 にアップロードする
  *  `.ipynb_checkpoint/` なども転送されるので注意
  * 返り値はファイル群を配置した prefix を返す
* fit の引数には S3 URI の prefix 以下を入れればその prefix 以下のファイルをトレーニングインスタンスに転送する
  * 転送先は変わらず環境変数 `SM_CHANNEL_TRAINING` が示すディレクトリ

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
input_s3_uri = sagemaker.session.Session().upload_data(path='./data/1-2-2/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='training/1-2-2')
print(input_s3_uri)
estimator = TensorFlow(
    entry_point='./src/1-2-2/calc.py',
    py_version='py38', 
    framework_version='2.7.1',
    instance_count=1,
    instance_type='local',
    role=sagemaker.get_execution_role()
)
estimator.fit(input_s3_uri)

### 1-2-3. 複数のディレクトリをトレーニングに持ち込む
#### 持ち込むデータの確認
`fold1` と `fold2` という 2 つのディレクトリにそれぞれ算数の演算データを配置

In [ ]:
FILE_NAME = './data/1-2-3/fold1/calc.txt'
print(f'check {FILE_NAME}')
with open(FILE_NAME) as f:
    print(f.read())

FILE_NAME = './data/1-2-3/fold2/calc.txt'
print(f'check {FILE_NAME}')
with open(FILE_NAME) as f:
    print(f.read())

##### 書き換えたい方向け

In [ ]:
# text1 = """1+2
# 3+4"""
# text2 = """5+6
# 7+8"""
# with open('./data/1-2-3/fold1/calc.txt','wt') as f:
#     f.write(text1)
# with open('./data/1-2-3/fold2/calc.txt','wt') as f:
#     f.write(text2)

#### 実行コードの確認
複数の S3 データソースをトレーニングインスタンスに持ち込む場合は、`SM_CHANNEL_{CHANNEL 名}` という環境変数が示すディレクトリに配置される。  
CHANNEL 名についてはユーザが `fit` メソッドを呼ぶときに決められる。  
`SM_CHANNELS` という環境変数に CHANNEL 名一覧が格納されるので、どのような CHANNEL 名が来てもいいように対応しておく


In [ ]:
!pygmentize ./src/1-2-3/calc.py

#### 持ち込むデータを S3 にアップロード
ディレクトリごとに `upload_data` メソッドを実行する。

In [ ]:
import sagemaker
fold1_input_s3_uri = sagemaker.session.Session().upload_data(path='./data/1-2-3/fold1/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='training/1-2-3/fold1')
fold2_input_s3_uri = sagemaker.session.Session().upload_data(path='./data/1-2-3/fold2/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='training/1-2-3/fold2')
print(fold1_input_s3_uri, fold2_input_s3_uri)

#### トレーニングジョブの実行
複数のデータをトレーニングインスタンスに持ち込む場合は、`fit` メソッドの引数に dict 型で持ち込むデータを入力する。  
dict のキーはトレーニングインスタンスの channel 名になり、値は持ち込む S3 のデータを入力する。

In [ ]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
    entry_point='./src/1-2-3/calc.py',
    py_version='py38', 
    framework_version='2.7.1',
    instance_count=1,
    instance_type='local',
    role=sagemaker.get_execution_role()
)
estimator.fit({
    'fold1':fold1_input_s3_uri,
    'fold2':fold2_input_s3_uri,
})

## 1-3. アーティファクトを S3 に転送する
### 用意したデータの確認
同じディレクトリに csv を 2 つ用意する。

In [ ]:
FILE_NAME = './data/1-3/data1.csv'
print(f'check {FILE_NAME}')
with open(FILE_NAME) as f:
    print(f.read())

FILE_NAME = './data/1-3/data2.csv'
print(f'check {FILE_NAME}')
with open(FILE_NAME) as f:
    print(f.read())

#### 書き換えたい方向け

In [ ]:
# text1="""1,2
# 3,4"""
# text2="""5,6
# 7,8"""
# with open('./data/1-3/data1.csv','wt') as f:
#     f.write(text1)
# with open('./data/1-3/data2.csv','wt') as f:
#     f.write(text2)

### 用意したコードの確認
用意したデータを連結する処理を記述。
連結したデータは、環境変数 `SM_MODEL_DIR`(=`/opt/ml/output/`) が示すディレクトリに配置するか、環境変数 `SM_OUTPUT_DATA_DIR`(=`/opt/ml/output/data/`) が示すディレクトリに保存すると自動でトレーニング終了時に S3 に転送される。
`SM_MODEL_DIR` と `SM_OUTPUT_DATA_DIR` の使い分けは、`SM_MODEL_DIR` は機械学習のモデルを保存し、それ以外(中間生成物など)は `SM_OUTPUT_DATA_DIR` に保存する。  
`SM_MODEL_DIR` に保存したモデルは Amazon SageMaker Hosting でそのまま推論環境のホスティングに使うことができる。

In [ ]:
!pygmentize ./src/1-3/concat.py

### トレーニングジョブの実行
アーティファクトを連携するのに estimator 側は特に特記することはなし

In [ ]:
import sagemaker
input_s3_uri = sagemaker.session.Session().upload_data(path='./data/1-3/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='training/1-3')
print(input_s3_uri)
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
    entry_point='./src/1-3/concat.py',
    py_version='py38', 
    framework_version='2.7.1',
    instance_count=1,
    instance_type='local',
    role=sagemaker.get_execution_role()
)
estimator.fit(input_s3_uri)

### ジョブの詳細確認
保存した model.tar.gz や、ジョブの詳細は `describe` メソッドで確認できる

In [ ]:
estimator.latest_training_job.describe()

### アーティファクトの確認
アーティファクトが保存されているか、download_data メソッドを使って手元にダウンロードして確認する。  
tar.gz に固められるため、解凍して中身を確認する

In [ ]:
import tarfile
prefix = estimator.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts'].replace(f's3://{sagemaker.session.Session().default_bucket()}/','')
sagemaker.session.Session().download_data('./', sagemaker.session.Session().default_bucket(), key_prefix=prefix)
with tarfile.open('./model.tar.gz', 'r') as f:
    f.extractall()
with open('./output.csv','rt') as f:
    print(f.read())

## 1-4. ライブラリの追加
マネジメントコンテナには機械学習で必要なほとんどのライブラリがあるが、追加したいケースもある。コンテナイメージをいじらずにトレーニングジョブ実行時にライブラリをインストールする仕組みがあるのでそのやり方を以下に記載。
### 用意したコード
トレーニングコードと同じディレクトリに pip で利用する `requirements.txt` を準備しておく。

In [ ]:
FILE_NAME = './src/1-4/requirements.txt'
print(f'check {FILE_NAME}')
with open(FILE_NAME) as f:
    print(f.read())

FILE_NAME = './src/1-4/bs4_version_check.py'
print(f'check {FILE_NAME}')    
!pygmentize {FILE_NAME}

#### 書き換えたい方向け

In [ ]:
# requirements = """beautifulsoup4==4.9.3"""
# code = """import bs4
# print(f'BeautifulSoup version is {bs4.__version__}.')
# exit()"""
# with open('./src/1-4/requirements.txt','wt') as f:
#     f.write(requirements)
# with open('./src/1-4/bs4_version_check.py','wt') as f:
#     f.write(code)

### トレーニングジョブを実行
`source_dir` 引数で `requirements.txt` とトレーニングコードを配置したディレクトリを指定する。  
トレーニングコードも `requirements.txt` も `source_dir` のルートに配置する必要があるので注意。  
また、`source_dir` で指定したファイルは全てトレーニングインスタンスに転送され、そのディレクトリ(環境変数 PWD で指定される `/opt/ml/code`)がカレントディレクトリになるので、自作モジュール(`model.py`など)も同ディレクトリ以下に配置しておくとトレーニングインスタンスで使用できる。
標準出力をよく見るとトレーニングコード実行前に pip で `requirements.txt` の内容をインストールしていることがわかる。

In [ ]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
    entry_point='bs4_version_check.py',
    source_dir='./src/1-4',
    py_version='py38', 
    framework_version='2.7.1',
    instance_count=1,
    instance_type='local',
    role=sagemaker.get_execution_role()
)
estimator.fit()

## 1-5 ハイパーパラメータをコードの外部から入力
トレーニングコードの外からハイパーパラメータを与えて、ハイパーパラメータの試行錯誤を行いたいケースがあるが、Amazon SageMaker Training はその機能を当然サポートしている。  
### 用意したコード
ハイパーパラメータを Amazon SageMaker Training で外部から与える場合、トレーニングコード側がその値を取得する方法は 2 つありどちらを使っても良い。  
1 つ目はコマンドライン引数で受け取れるので以下の例のように argparse などを用いて、parse して利用する。  
2 つめは環境変数 SM_HPS の値を読み込む。JSON 形式の文字列で格納されているので、`hps = json.loads(os.environ.get('SM_HPS'))` のように書くと `hps` 変数に一度に格納することができる。  
いずれの方法でもデフォルト値を設定したほうが便利なので、add_argument メソッドに defalut 引数を入れるか、hps.setdefault メソッドを利用する、のが良い。  
以下は `argparse` を使った例で、数字を 2 つ、演算子を 1 つ与えて、足し算か引き算を行うコード。`model_dir` 引数は、必ずコマンドライン引数に入るため、parse しないとエラーで落ちるために入れている。

In [ ]:
!pygmentize ./src/1-5/hp_calc.py

In [ ]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
    entry_point='./src/1-5/hp_calc.py',
    py_version='py38', 
    framework_version='2.7.1',
    instance_count=1,
    instance_type='local',
    role=sagemaker.get_execution_role(),
    hyperparameters={
        'first-num':5,
        'second-num':2,
        'operator':'m'
    }
)
estimator.fit()

## トレーニングジョブの標準出力
* 標準出力やメトリクスは Amazon CloudWatch に自動で連携される
* ここでは Amazon CloudWatch Logs で最後に実行した標準出力の内容を表示する。
* メトリクスについてはマネジメントコンソールで確認するのが簡単。

In [ ]:
import boto3
logs = boto3.client('logs')
log_group_name = '/aws/sagemaker/TrainingJobs'
latest_logstream_name = logs.describe_log_streams(
    logGroupName=log_group_name,
    orderBy='LastEventTime',
    descending=True
)['logStreams'][0]['logStreamName']
logs.get_log_events(
    logGroupName=log_group_name,
    logStreamName=latest_logstream_name,
)